# SKEMA Experiment

Testing out SKEMA using a pet example from [this paper on a compartmental COVID-19 Model](https://www.sciencedirect.com/science/article/pii/S0960077921000424). `./model` contains the Python implementation of this model which was copied from [this Jataware repository](https://github.com/jataware/COVID-19_Projections). The original script reads from two JSON files but for this SKEMA experiment, they have been combined into one `options.py`.

This notebook is assumes you follow the installation instructions from the [TA1 December Demo Repo](https://github.com/ml4ai/ASKEM-TA1-SKEMA-Demo-2022-12).

In [1]:
import os
import json
from pprint import pprint
from automates.utils.fold import dictionary_to_gromet_json, del_nulls
from skema.text_reading.mention_linking.gromet_linker import align_and_link
from automates.utils.script_functions import process_file_system
from automates.program_analysis.JSON2GroMEt.json2gromet import json_to_gromet

MODEL_DIR = os.path.abspath("./model")
RESULTS_DIR = os.path.abspath("./demo_results")
SYSTEM_FILES_PATH= os.path.join(MODEL_DIR, "filepaths.txt")
GROMET_PATH = os.path.join(RESULTS_DIR, "test-auto.json")


os.makedirs(RESULTS_DIR, exist_ok=True)


## Code2FN

I tried parsing code into a Function Network below. 

First, I used `process_file_system` which 
yielded [this JSON](./test--Gromet-FN-auto.json) but upon updating, that step broke. 

In [2]:
COLLECTION_NAME = os.path.join(RESULTS_DIR, "test")


process_file_system(COLLECTION_NAME, MODEL_DIR, SYSTEM_FILES_PATH)

/home/five/askem-experiments/skema/model/main.py
found numpy
found scipy.integrate
found matplotlib.pyplot
found options
found options


TypeError: 'NoneType' object is not iterable

Second, I tried processing the JSON above using `json_to_gromet`, but it failed. It was commented out during the December Demo so I expect that it has issues.

In [3]:
# DOESN't work
chime_module_collection = json_to_gromet(COLLECTION_NAME+"--Gromet-FN-auto.json")
pprint(chime_module_collection)

KeyError: 'fn'

Since the above function didn't work, I tried to find a function that would just take some Python (not a whole filesystem) and return a gromet. `py2grfn` seemed like what I wanted from the name, but I couldn't import it without module error.

In [4]:
# This also doesn't seem to work
from automates.program_analysis.Py2GrFN import py2grfn

ModuleNotFoundError: No module named 'cast'

In [5]:
with open(COLLECTION_NAME+"--Gromet-FN-auto.json", 'r') as f:
    chime_module_collection = json.load(f)
#pprint()

## [SKIP] METAL

cannot use METAL because the following objects are not available. 
```python
COMMENTS_PATH = os.path.join(RESULTS_DIR, "test-comments.json")
EXTRACTIONS_PATH = os.path.join(RESULTS_DIR, "test-extractions.json")
EMBEDDINGS_PATH = os.path.join(RESULTS_DIR, "embeddings.kv")
```

## SKEMA Server and FN Graph Service

The IFrame below doesn't render but I believe it's a user error on my part. Still, if I try to visit the documentation in my browser, it doesn't work. The server is still running because the `ping` endpoint works.

In [6]:
from IPython.display import IFrame
IFrame('http://localhost:8080/api-docs/', width=700, height=350)

In [7]:
import json
from requests import get, put, post, delete
#from graphviz import Source
BASE_URL="http://localhost:8080"

In [8]:
get(f"{BASE_URL}/ping").text

'The SKEMA Rust web services are running.'

The generated JSON doesn't work. I believe it's because the `json_to_gromet` step failed which didn't turn the list `modules` in to a Function Network. In the December Demo, a different file is used I think for the reason that the `json_to_gromet` step wasn't working.

In [9]:
from requests import post
with open(os.path.join(os.path.abspath(".."),'test--Gromet-FN-auto.json')) as f:
    r = post(f"{BASE_URL}/models", json=json.loads(f.read()))
    if r.status_code == 200:
        MODEL_ID = r.json()
        print(f"Model ID: {MODEL_ID}")
    else:
        print(r.content.decode())

Json deserialize error: missing field `fn` at line 1 column 126693
